# Logistic Regression

In [1]:
import numpy as np
import pandas as pd

In [2]:
test_file = r"D:\PDFs\Edvancer Eduventures\Certified Machine Learning Expert\Data\rg_test.csv"
train_file = r"D:\PDFs\Edvancer Eduventures\Certified Machine Learning Expert\Data\rg_train.csv"

bd_train=pd.read_csv(train_file)
bd_test=pd.read_csv(test_file)

In [3]:
bd_test['Revenue.Grid']=np.nan
bd_train['data']='train'
bd_test['data']='test'
bd_test=bd_test[bd_train.columns]
bd_all=pd.concat([bd_train,bd_test],axis=0)

In [4]:
bd_all['Revenue.Grid'].value_counts()

Revenue.Grid
2.0    7261
1.0     863
Name: count, dtype: int64

In [5]:
list(zip(bd_all.columns,bd_all.dtypes,bd_all.nunique()))

[('REF_NO', dtype('int64'), 10155),
 ('children', dtype('O'), 5),
 ('age_band', dtype('O'), 13),
 ('status', dtype('O'), 5),
 ('occupation', dtype('O'), 9),
 ('occupation_partner', dtype('O'), 9),
 ('home_status', dtype('O'), 5),
 ('family_income', dtype('O'), 13),
 ('self_employed', dtype('O'), 2),
 ('self_employed_partner', dtype('O'), 2),
 ('year_last_moved', dtype('int64'), 95),
 ('TVarea', dtype('O'), 14),
 ('post_code', dtype('O'), 10040),
 ('post_area', dtype('O'), 2039),
 ('Average.Credit.Card.Transaction', dtype('float64'), 1411),
 ('Balance.Transfer', dtype('float64'), 2183),
 ('Term.Deposit', dtype('float64'), 1419),
 ('Life.Insurance', dtype('float64'), 3111),
 ('Medical.Insurance', dtype('float64'), 1589),
 ('Average.A.C.Balance', dtype('float64'), 2223),
 ('Personal.Loan', dtype('float64'), 1760),
 ('Investment.in.Mutual.Fund', dtype('float64'), 2470),
 ('Investment.Tax.Saving.Bond', dtype('float64'), 832),
 ('Home.Loan', dtype('float64'), 884),
 ('Online.Purchase.Amount'

In [6]:
# REF_NO,post_code , post_area  : drop 
# children : Zero : 0 , 4+ : 4 and then convert to numeric
# age_band : dummies 
# status , occupation , occupation_partner , home_status,family_income : dummies
# self_employed, ` : dummies
# TVArea , Region , gender : dummies
# Revenue Grid : 1,2 : 1,0

In [7]:
bd_all.drop(['REF_NO','post_code','post_area'],axis=1,inplace=True)

In [8]:
bd_all['children']=np.where(bd_all['children']=='Zero',0,bd_all['children'])
bd_all['children']=np.where(bd_all['children'][:1]=='4',4,bd_all['children'])
bd_all['children']=pd.to_numeric(bd_all['children'],errors='coerce')

In [9]:
bd_all['Revenue.Grid']=(bd_all['Revenue.Grid']==1).astype(int)

In [10]:
cat_vars=bd_all.select_dtypes(['object']).columns

In [11]:
cat_vars

Index(['age_band', 'status', 'occupation', 'occupation_partner', 'home_status',
       'family_income', 'self_employed', 'self_employed_partner', 'TVarea',
       'gender', 'region', 'data'],
      dtype='object')

In [12]:
for col in cat_vars[:-1]:
    dummy=pd.get_dummies(bd_all[col],drop_first=True,prefix=col)
    bd_all=pd.concat([bd_all,dummy],axis=1)
    del bd_all[col]
    print(col)
del dummy

age_band
status
occupation
occupation_partner
home_status
family_income
self_employed
self_employed_partner
TVarea
gender
region


In [13]:
bd_all.shape

(10155, 96)

In [14]:
bd_all.isnull().sum()

children                           19
year_last_moved                     0
Average.Credit.Card.Transaction     0
Balance.Transfer                    0
Term.Deposit                        0
                                   ..
region_South East                   0
region_South West                   0
region_Unknown                      0
region_Wales                        0
region_West Midlands                0
Length: 96, dtype: int64

In [15]:
bd_all.loc[bd_all['children'].isnull(),'children']=bd_all.loc[bd_all['data']=='train','children'].mean()

In [16]:
bd_train=bd_all[bd_all['data']=='train']
del bd_train['data']
bd_test=bd_all[bd_all['data']=='test']
bd_test.drop(['Revenue.Grid','data'],axis=1,inplace=True)

C:\Users\Siddhant Srivastava\AppData\Local\Temp\ipykernel_8412\3423853692.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bd_test.drop(['Revenue.Grid','data'],axis=1,inplace=True)


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [18]:
params={'class_weight':['balanced',None],
        'penalty':['l1','l2'],
        'C':np.linspace(0.01,1000,10)}

In [19]:
model=LogisticRegression(fit_intercept=True)

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
grid_search= GridSearchCV(model,param_grid=params,cv=5,scoring="roc_auc")

In [22]:
x_train=bd_train.drop('Revenue.Grid',axis=1)
y_train=bd_train['Revenue.Grid']

In [23]:
grid_search.fit(x_train,y_train)

C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.0000e-02, 1.1112e+02, 2.2223e+02, 3.3334e+02, 4.4445e+02,
       5.5556e+02, 6.6667e+02, 7.7778e+02, 8.8889e+02, 1.0000e+03]),
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2']},
             scoring='roc_auc')

In [24]:
grid_search.best_estimator_

LogisticRegression(C=444.45, class_weight='balanced')

In [25]:
logr=grid_search.best_estimator_

In [26]:
def report (results, n_top = 3):
    for i in range(1, n_top + 1):
        candidate = np.flatnonzero(results['rank_test_score'] == i)[0]
        
        print("model with rank: {0}".format(i))
        print("Mean Validation Score : {0:0.5f} (std: {1: 5f})".format(
        results['mean_test_score'][candidate],
        results['std_test_score'][candidate]))
        
        print("Parameters: {0}".format(results['params'][candidate]))
        print("")

In [27]:
report(grid_search.cv_results_,5)

model with rank: 1
Mean Validation Score : 0.95440 (std:  0.003982)
Parameters: {'C': 444.45, 'class_weight': 'balanced', 'penalty': 'l2'}

model with rank: 2
Mean Validation Score : 0.95437 (std:  0.004085)
Parameters: {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l2'}

model with rank: 3
Mean Validation Score : 0.95432 (std:  0.003996)
Parameters: {'C': 777.78, 'class_weight': 'balanced', 'penalty': 'l2'}

model with rank: 4
Mean Validation Score : 0.95429 (std:  0.003901)
Parameters: {'C': 1000.0, 'class_weight': 'balanced', 'penalty': 'l2'}

model with rank: 5
Mean Validation Score : 0.95427 (std:  0.003861)
Parameters: {'C': 555.56, 'class_weight': 'balanced', 'penalty': 'l2'}



In [28]:
logr.fit(x_train,y_train)

C:\Users\Siddhant Srivastava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=444.45, class_weight='balanced')

In [29]:
cutoffs=np.linspace(0.01,0.99,99)
cutoffs

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [30]:
train_score=logr.predict_proba(x_train)[:,1]
real=y_train

In [31]:
train_score>0.2

array([False, False, False, ...,  True, False, False])

In [32]:
KS_all=[]

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP
      
    KS=(TP/P)-(FP/N)
    
    
    KS_all.append(KS)

# try out what cutoffs you get when you use F_beta scores with different values of betas [0.5 , 5]
# beta < 1 : you will get cutoff , which is high ( favours precision)
# beta > 1 : you will get cutoff , which is low (favours precision )

In [33]:
mycutoff=cutoffs[KS_all==max(KS_all)][0]
mycutoff

0.5

In [34]:
logr.intercept_

array([-0.00079367])

In [35]:
list(zip(x_train.columns,logr.coef_[0]))

[('children', 0.00048044317084954),
 ('year_last_moved', -0.001305683968625805),
 ('Average.Credit.Card.Transaction', 0.023091099358968886),
 ('Balance.Transfer', -0.005307050741010535),
 ('Term.Deposit', -0.020300443305885665),
 ('Life.Insurance', 0.01285154981797748),
 ('Medical.Insurance', -0.009835830038317415),
 ('Average.A.C.Balance', -0.005330621297885524),
 ('Personal.Loan', -0.033151930381983226),
 ('Investment.in.Mutual.Fund', -0.0014931477966737038),
 ('Investment.Tax.Saving.Bond', 0.09198078344370406),
 ('Home.Loan', -0.07269523197671358),
 ('Online.Purchase.Amount', 0.05300983303833148),
 ('Investment.in.Commudity', 9.94958358768863e-05),
 ('Investment.in.Equity', 0.005380481083767485),
 ('Investment.in.Derivative', 0.009173315350054575),
 ('Portfolio.Balance', -8.305858319267127e-05),
 ('age_band_22-25', 3.933657689876788e-05),
 ('age_band_26-30', 0.0004979489196785298),
 ('age_band_31-35', 0.00031481643930814443),
 ('age_band_36-40', -7.988590892834086e-05),
 ('age_band_

In [36]:
test_score=logr.predict_proba(bd_test)[:,1]
pd.DataFrame(test_score).to_csv("mysubmission.csv",index=False)

In [37]:
test_classes=(test_score>mycutoff).astype(int)

In [38]:
pd.DataFrame(test_classes).to_csv("mysubmission.csv",index=False)